# qBraid-QIR: Rigetti QVM simulation example

Install necessary dependencies

In [ ]:
%pip install qbraid-qir cirq-core azure-quantum --quiet

Develop program with Cirq

In [ ]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CNOT(q0, q1), cirq.measure(q0, q1))

print(circuit)

Compile program to QIR

In [ ]:
from qbraid_qir.cirq import cirq_to_qir

module = cirq_to_qir(circuit, name="bell")

print(str(module))

Submit program to Rigetti  via Azure Quantum Cloud

<div class="alert alert-block alert-info">
    <b>Note:</b> You will have to open a terminal and run <code>az login</code> to make sure your Azure credentials are stored.
</div>

In [ ]:
import azure.quantum
from azure.identity import AzureCliCredential

workspace = azure.quantum.Workspace(
    subscription_id="<subscription-id>",
    resource_group="AzureQuantum",
    name="qir-demo",
    location="westus",
    credential=AzureCliCredential(),
)

In [ ]:
job = azure.quantum.Job.from_input_data(
    workspace=workspace,
    name="qir-demo",
    provider_id="rigetti",
    target="rigetti.sim.qvm",
    input_data_format="qir.v1",
    output_data_format="microsoft.quantum-results.v1",
    input_data=module.bitcode,
    input_params={"shots": 1, "entryPoint": "main", "arguments": []},
)

info = job.details.as_dict()

In [ ]:
job.get_results()